<a href="https://colab.research.google.com/github/hyuna0926/cp2_phase2/blob/main/2%EC%9B%94%202%EC%9D%BC/model_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 필요 라이브러리 들고 오기

In [32]:
! pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from scipy import sparse
from scipy.sparse import csr_matrix
from tqdm.notebook import tqdm
import numpy as np

from implicit.als import AlternatingLeastSquares as ALS
import implicit
import random
import os

# 데이터 합치기

In [36]:
trans = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/transaction_new.parquet')
customer = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/customer.parquet')
product = pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/product_final.parquet')

In [46]:
trans['created_at']=trans['created_at'].apply(lambda x: x[:19].replace('T',' '))

In [48]:
trans_c = trans[['created_at','customer_id', 'product_id']]

In [50]:
trans_c.sort_values('created_at')

,created_at,customer_id,product_id
1254461,2016-06-30 23:18:44,74089,33314
1201589,2016-07-01 02:53:26,6183,16061
1201590,2016-07-01 02:53:26,6183,7588
1201721,2016-07-01 09:45:51,16228,47650
1254460,2016-07-01 22:32:45,73773,32185
...,...,...,...
985626,2022-07-31 23:58:50,81975,50710
985568,2022-07-31 23:58:53,19551,17773
985766,2022-07-31 23:59:03,22285,27404
985569,2022-07-31 23:59:19,19551,25189


In [52]:
customer['c_idx']=customer.index

In [57]:
customer['c_idx']=customer['c_idx'].astype('int32')
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   customer_id         100000 non-null  int32   
 1   first_name          100000 non-null  category
 2   last_name           100000 non-null  category
 3   username            100000 non-null  object  
 4   email               100000 non-null  object  
 5   gender              100000 non-null  category
 6   birthdate           100000 non-null  category
 7   device_type         100000 non-null  category
 8   device_id           100000 non-null  object  
 9   device_version      100000 non-null  category
 10  home_location_lat   100000 non-null  int8    
 11  home_location_long  100000 non-null  uint8   
 12  home_location       100000 non-null  category
 13  home_country        100000 non-null  category
 14  first_join_date     100000 non-null  category
 15  c_idx             

In [58]:
# customer.to_parquet('customer.parquet')

In [59]:
tran_cus = pd.merge(trans_c, customer[['c_idx','customer_id']], on='customer_id')

In [60]:
tran_cus.head()

,created_at,customer_id,product_id,c_idx
0,2018-07-29 15:22:01,5868,54728,38400
1,2018-07-30 12:40:22,4774,16193,25035
2,2018-09-15 11:51:17,4774,53686,25035
3,2018-11-01 11:23:48,4774,20228,25035
4,2018-12-18 11:20:30,4774,55220,25035


In [61]:
trans_cus_pro = pd.merge(tran_cus, product[['p_idx','product_id']],on='product_id')

In [65]:
trans_cus_pro.to_parquet('model_data.parquet')